<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [1]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'João Vitor Choueri Branco, 21.01075-7' #@param {type:"string"}






In [131]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'
import pandas as pd
import numpy as np

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [132]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [31]:
df_q1 = df.copy()
df_q1.host_name.value_counts()[:5]

host_name
Mark            7
Sonder (NYC)    7
Andrew          6
David           6
Mike            5
Name: count, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [25]:
df_neighbourhood_count = df_q1.neighbourhood.value_counts().reset_index()
df_neighbourhood_count.columns = ['neighbourhood', 'ct']
df_neighbourhood_count = df_neighbourhood_count[df_neighbourhood_count["ct"] > 20]
nb_list = df_neighbourhood_count.neighbourhood.to_list()

preco_medio = df_q1[df_q1.neighbourhood.isin(nb_list)].price.mean()

In [29]:
df_neighbourhood_count

neighbourhood  ct
0   Bedford-Stuyvesant  69
1         Williamsburg  65
2               Harlem  50
3             Bushwick  41
4       Hell's Kitchen  31
5              Midtown  30
6         East Village  29
7        Crown Heights  28
8   Washington Heights  25
9      Upper West Side  23
10     Upper East Side  22

In [26]:
...

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [71]:
# outliers
df_man = df[df.neighbourhood_group == 'Brooklyn']
IQR = df_man.price.quantile(0.75) - df_man.price.quantile(0.25)
inf = df_man.price.quantile(0.25) - 1.5 * IQR
sup = df_man.price.quantile(0.75) + 1.5 * IQR
IQR

np.float64(86.0)

In [72]:
num_tot_man = df_man.shape[0]
num_out_man = df_man[(df_man.price < inf) | (df_man.price > sup)].shape[0]
perc_Brooklyn = num_out_man / num_tot_man * 100
perc_Brooklyn

5.757575757575758

In [73]:
...

print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:.2f} %')


Percentuais de outliers de preços, Brooklyn: 5.76 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [58]:
df.head(1)

id                                name  host_id   host_name  \
0  20913  Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   

  neighbourhood_group neighbourhood  latitude  longitude        room_type  \
0            Brooklyn  Williamsburg  40.70984  -73.95775  Entire home/apt   

   price  minimum_nights  number_of_reviews last_review  reviews_per_month  \
0    100               5                168  2018-07-22               1.57   

   calculated_host_listings_count  availability_365  
0                               1                 0

In [59]:
# prompt: crie uma funcao que determina as seguintes regras:
# Imóveis com availability_365 > 90 recebem +10 pontos enquanto imóveis com availability_365 < 5 recebem -5 pontos;
# Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
# Imóveis com preço abaixo da média de seu neighbourhood recebem +5 pontos.

def atribuir_score(row):
    score = 0

    # Regra 1
    if row['availability_365'] > 90:
        score += 10
    elif row['availability_365'] < 5:
        score -= 5

    # Regra 2
    if row['number_of_reviews'] >= 10:
        score += 5 * (row['number_of_reviews'] // 10)
    elif row['number_of_reviews'] == 0:
        score -= 5

    # Regra 3
    media_bairro = df[df['neighbourhood'] == row['neighbourhood']]['price'].mean()
    if row['price'] < media_bairro:
        score += 5

    return score

df['score'] = df.apply(atribuir_score, axis=1)
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  score  
0               1.57                               1                 0     80  
1               1.46                               2               137     90  
2               3.97                               3               182    210  
3               0.91                               2               286     50  
4               0.43                               2                59     25

In [60]:
...

print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  10955


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [61]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes.head(2)

id   type   value
0  35907127  taxes  1315.0
1    804815  taxes     NaN

In [90]:
df_taxes[df_taxes.id == 35907127]

id   type   value
0    35907127  taxes  1315.0
764  35907127  fines     NaN

In [95]:
df_imposto = df_taxes[df_taxes["type"] == "taxes"]
df_multa = df_taxes[df_taxes["type"] == "fines"]
print(
    "shape df_imposto: ", df_imposto.shape,
    "\nshape df_multa: ", df_multa.shape
)

shape df_imposto:  (764, 3) 
shape df_multa:  (764, 3)


In [107]:
df_full = df.merge(df_imposto, on='id', how='inner')
df_full = df_full.merge(df_multa, on='id', how='inner')
print(df_full.shape)
df_full.columns = ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'score', 'type_1', 'taxes', 'type_2', 'fines']
df_full = df_full.drop(["type_1", "type_2"], axis=1)
df_full.head(2)

(764, 21)


id                                name  host_id   host_name  \
0  20913  Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
1  25235   Large 2 Bedroom Great for Groups!    87773      Shelly   

  neighbourhood_group       neighbourhood  latitude  longitude  \
0            Brooklyn        Williamsburg  40.70984  -73.95775   
1            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   

   reviews_per_month  calculated_host_listings_count  availability_365  score  \
0               1.57                               1                 0     80   
1               1.46                               2               137     90   

    taxes  fines  
0   789.0  796.0  
1  1779.0    NaN

In [108]:
# Considerando fines e taxes como 0
df_full = df_full.fillna(0)
df_full.head(2)

id                                name  host_id   host_name  \
0  20913  Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
1  25235   Large 2 Bedroom Great for Groups!    87773      Shelly   

  neighbourhood_group       neighbourhood  latitude  longitude  \
0            Brooklyn        Williamsburg  40.70984  -73.95775   
1            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   

   reviews_per_month  calculated_host_listings_count  availability_365  score  \
0               1.57                               1                 0     80   
1               1.46                               2               137     90   

    taxes  fines  
0   789.0  796.0  
1  1779.0    0.0

In [109]:
df_full.groupby("neighbourhood_group")[["taxes", "fines"]].agg({
    'taxes': 'mean',
    'fines': 'mean'
})


taxes       fines
neighbourhood_group                         
Bronx                 851.052632  126.473684
Brooklyn              858.276873  314.472313
Manhattan             888.844633  314.980226
Queens                804.166667  322.769231
Staten Island        1157.500000  581.333333

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [178]:
df.head(2)

id                                name  host_id   host_name  \
0  20913  Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
1  25235   Large 2 Bedroom Great for Groups!    87773      Shelly   

  neighbourhood_group       neighbourhood  latitude  longitude  \
0            Brooklyn        Williamsburg  40.70984  -73.95775   
1            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137

In [213]:
df_q6 = df.copy()
df_q6 = df_q6[["neighbourhood_group", "neighbourhood", "price"]].sort_values(by=["neighbourhood_group", "neighbourhood"]).reset_index(drop=True)
df_q6

neighbourhood_group      neighbourhood  price
0                 Bronx            Belmont     29
1                 Bronx  Claremont Village     67
2                 Bronx          Concourse     40
3                 Bronx           Edenwald     90
4                 Bronx           Edenwald     80
..                  ...                ...    ...
799       Staten Island        South Beach     20
800       Staten Island        South Beach    100
801       Staten Island         St. George    130
802       Staten Island          Todt Hill    135
803       Staten Island      Tompkinsville     59

[804 rows x 3 columns]

In [214]:
# prompt: Crie uma função de inserir linha, com um exemplo de dataframe mock

def insert_row(df_var, row_number, row_data):
    return pd.concat([df_var.iloc[:row_number], pd.DataFrame(row_data, index=[1]), df_var.iloc[row_number:]]).reset_index(drop=True)

# Example usage with mock DataFrame
data = {'col1': [1, 2, 3], 'col2': ['A', 'B', 'C']}
df_mock = pd.DataFrame(data)
print(df_mock)

# Insert a new row at index 1
new_row_data = {'col1': 10, 'col2': 'X'}
df_updated = insert_row(df_mock, 1, new_row_data)
print(df_updated)



   col1 col2
0     1    A
1     2    B
2     3    C
   col1 col2
0     1    A
1    10    X
2     2    B
3     3    C


In [215]:
# criando colunas a serem manipuladas
df_q6["line_id"] = 1
df_q6["page_id"] = 1
df_q6

# variaveis
nb_anal = "Bronx"
total_pages = 1
line_id_count = 0

# criando a paginação
for i, infos in df_q6.iterrows():

  # se chegou no final de um nb_group
  if infos["neighbourhood_group"] != nb_anal:
    nb_anal = infos["neighbourhood_group"]
    total_pages += 1
    line_id_count = 1
    df_q6.loc[i, "page_id"] = total_pages
    df_q6.loc[i, "line_id"] = line_id_count

  # se chegou no final de uma página
  elif line_id_count == 30:
    total_pages += 1
    line_id_count = 1
    df_q6.loc[i, "page_id"] = total_pages
    df_q6.loc[i, "line_id"] = line_id_count

  # se ainda está na paginação
  else:
    line_id_count += 1
    df_q6.loc[i, "page_id"] = total_pages
    df_q6.loc[i, "line_id"] = line_id_count



In [201]:
# new_row_data = {
#           "neighbourhood_group": 1,
#           "neighbourhood": "MEAN",
#           "price": 1,
#           "line_id":  1,
#           "page_id": 1
#       }
# df_q6 = insert_row(df_q6, 1, new_row_data)
# print(df_q6)

    neighbourhood_group      neighbourhood  price  line_id  page_id
0                 Bronx            Belmont     29        1        1
1                     1               MEAN      1        1        1
2                 Bronx  Claremont Village     67        2        1
3                 Bronx          Concourse     40        3        1
4                 Bronx           Edenwald     90        4        1
..                  ...                ...    ...      ...      ...
800       Staten Island        South Beach     20        3       29
801       Staten Island        South Beach    100        4       29
802       Staten Island         St. George    130        5       29
803       Staten Island          Todt Hill    135        6       29
804       Staten Island      Tompkinsville     59        7       29

[805 rows x 5 columns]


In [218]:
df_q6

neighbourhood_group      neighbourhood  price  line_id  page_id
0                 Bronx            Belmont     29        1        1
1                 Bronx  Claremont Village     67        2        1
2                 Bronx          Concourse     40        3        1
3                 Bronx           Edenwald     90        4        1
4                 Bronx           Edenwald     80        5        1
..                  ...                ...    ...      ...      ...
799       Staten Island        South Beach     20        3       29
800       Staten Island        South Beach    100        4       29
801       Staten Island         St. George    130        5       29
802       Staten Island          Todt Hill    135        6       29
803       Staten Island      Tompkinsville     59        7       29

[804 rows x 5 columns]

In [211]:
# Agrupando para ter uma Serie de média
df_report = df_q6.groupby(["neighbourhood_group"])["price"].mean().reset_index()
nb_dict = dict(zip(df_report['neighbourhood_group'], df_report['price']))
nb_dict

{'Bronx': 71.5,
 'Brooklyn': 128.0878787878788,
 'Manhattan': 196.1576086956522,
 'Queens': 92.22784810126582,
 'Staten Island': 89.42857142857143}

In [216]:
# # inserindo novas linhas
# # criando criando função para o dataframe, por conta do problema de alterar o dataframe enquanto faz um for por index
# def add_mean_2_df(df, nb_2_stop):
#   is_in_col = False

#   for i, infos in df.iterrows():

#     # se chegou no final de um nb_group
#     if infos["neighbourhood_group"] != nb_2_stop and is_in_col:
#       new_row_data = {
#           "neighbourhood_group": nb_2_stop,
#           "neighbourhood": "MEAN",
#           "price": nb_dict[nb_2_stop],
#           "line_id": infos["line_id"] + 1,
#           "page_id": infos["page_id"]
#       }
#       return insert_row(df, i + 1, new_row_data)

#     elif infos["neighbourhood_group"] == nb_2_stop:
#       print("fez_a_troca")
#       is_in_col = True

# for nb in nb_dict.keys()[:3]:
#   df_q6 = add_mean_2_df(df_q6, nb)

# df_q6

In [185]:
# # prompt: crie um dataframe de uma lina apenas, com as mesmas colunas de df_q6

# # Crie um DataFrame vazio com as mesmas colunas de df_q6
# df_uma_linha = pd.DataFrame(columns=df_q6.columns)

# # Crie uma nova linha com dados fictícios ou valores padrão
# nova_linha = pd.DataFrame({
#     'neighbourhood_group': ['Staten Island'],  # Substitua pelos valores desejados
#     'neighbourhood': ['MEAN'],
#     'price': [nb_dict["Staten Island"]],
#     'line_id': [8],
#     'page_id': [29]
# })

# # Concatene a nova linha ao DataFrame vazio
# df_uma_linha = pd.concat([df_uma_linha, nova_linha], ignore_index=True)

# # Imprima o DataFrame resultante
# df_uma_linha

# # concatenando
# df_report = pd.concat([df_q6, df_uma_linha], ignore_index=True)
# df_report


AttributeError: 'NoneType' object has no attribute 'columns'

In [219]:
df_report = df_q6
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 15160


## SAMPLE (Não alterar linha de código abaixo)

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7

Q7


A preparação dos dados no CRISP-DM envolve várias tarefas essenciais.  
Primeiramente, a coleta dos dados, que pode vir de várias fontes, é fundamental.
Em seguida, a etapa de limpeza trata valores faltantes, outliers e inconsistências, garantindo a qualidade dos dados.  
A transformação dos dados inclui conversão de tipos, normalização e criação de novas variáveis para melhorar a análise.
A integração de diferentes fontes de dados é crucial para uma visão completa.
A redução de dimensionalidade, através de seleção ou extração de características, simplifica o modelo sem perda de informação relevante.
Por fim, a formatação dos dados para o modelo escolhido é essencial, garantindo compatibilidade e otimização do processo.